In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'


In [3]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

In [4]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [5]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [6]:
# Device setting
D = D.to(device)
G = G.to(device)

In [7]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [8]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 0.0462, g_loss: 4.1220, D(x): 0.99, D(G(z)): 0.04
Epoch [0/200], Step [400/600], d_loss: 0.0843, g_loss: 5.9248, D(x): 0.98, D(G(z)): 0.06
Epoch [0/200], Step [600/600], d_loss: 0.0274, g_loss: 5.4303, D(x): 0.99, D(G(z)): 0.02
Epoch [1/200], Step [200/600], d_loss: 0.0530, g_loss: 5.7069, D(x): 0.97, D(G(z)): 0.02
Epoch [1/200], Step [400/600], d_loss: 0.3705, g_loss: 2.8343, D(x): 0.92, D(G(z)): 0.17
Epoch [1/200], Step [600/600], d_loss: 0.3542, g_loss: 5.4087, D(x): 0.93, D(G(z)): 0.13
Epoch [2/200], Step [200/600], d_loss: 0.0673, g_loss: 5.4127, D(x): 0.96, D(G(z)): 0.02
Epoch [2/200], Step [400/600], d_loss: 0.2121, g_loss: 4.2913, D(x): 0.90, D(G(z)): 0.07
Epoch [2/200], Step [600/600], d_loss: 0.2440, g_loss: 4.6631, D(x): 0.94, D(G(z)): 0.07
Epoch [3/200], Step [200/600], d_loss: 0.7120, g_loss: 3.1701, D(x): 0.84, D(G(z)): 0.27
Epoch [3/200], Step [400/600], d_loss: 0.1743, g_loss: 3.5660, D(x): 0.98, D(G(z)): 0.13
Epoch [3/200], Step [

Epoch [30/200], Step [400/600], d_loss: 0.2897, g_loss: 4.0499, D(x): 0.88, D(G(z)): 0.08
Epoch [30/200], Step [600/600], d_loss: 0.5675, g_loss: 2.0908, D(x): 0.84, D(G(z)): 0.17
Epoch [31/200], Step [200/600], d_loss: 0.4552, g_loss: 4.0406, D(x): 0.82, D(G(z)): 0.06
Epoch [31/200], Step [400/600], d_loss: 0.3752, g_loss: 3.3907, D(x): 0.87, D(G(z)): 0.12
Epoch [31/200], Step [600/600], d_loss: 0.3679, g_loss: 3.4219, D(x): 0.90, D(G(z)): 0.11
Epoch [32/200], Step [200/600], d_loss: 0.4543, g_loss: 3.5868, D(x): 0.87, D(G(z)): 0.10
Epoch [32/200], Step [400/600], d_loss: 0.3910, g_loss: 4.1218, D(x): 0.89, D(G(z)): 0.14
Epoch [32/200], Step [600/600], d_loss: 0.2206, g_loss: 3.4411, D(x): 0.95, D(G(z)): 0.12
Epoch [33/200], Step [200/600], d_loss: 0.3025, g_loss: 2.4644, D(x): 0.91, D(G(z)): 0.11
Epoch [33/200], Step [400/600], d_loss: 0.3875, g_loss: 4.6271, D(x): 0.85, D(G(z)): 0.07
Epoch [33/200], Step [600/600], d_loss: 0.5140, g_loss: 3.2285, D(x): 0.90, D(G(z)): 0.20
Epoch [34/

Epoch [60/200], Step [600/600], d_loss: 0.4771, g_loss: 3.2391, D(x): 0.83, D(G(z)): 0.16
Epoch [61/200], Step [200/600], d_loss: 0.8996, g_loss: 2.4938, D(x): 0.68, D(G(z)): 0.10
Epoch [61/200], Step [400/600], d_loss: 0.5698, g_loss: 2.4576, D(x): 0.79, D(G(z)): 0.15
Epoch [61/200], Step [600/600], d_loss: 0.6364, g_loss: 2.5191, D(x): 0.78, D(G(z)): 0.20
Epoch [62/200], Step [200/600], d_loss: 0.6541, g_loss: 4.0148, D(x): 0.73, D(G(z)): 0.06
Epoch [62/200], Step [400/600], d_loss: 0.6863, g_loss: 2.6574, D(x): 0.80, D(G(z)): 0.22
Epoch [62/200], Step [600/600], d_loss: 0.6431, g_loss: 2.8655, D(x): 0.79, D(G(z)): 0.20
Epoch [63/200], Step [200/600], d_loss: 0.4249, g_loss: 3.6454, D(x): 0.82, D(G(z)): 0.10
Epoch [63/200], Step [400/600], d_loss: 0.5916, g_loss: 3.0979, D(x): 0.77, D(G(z)): 0.15
Epoch [63/200], Step [600/600], d_loss: 0.6205, g_loss: 2.4569, D(x): 0.84, D(G(z)): 0.24
Epoch [64/200], Step [200/600], d_loss: 0.5504, g_loss: 2.4610, D(x): 0.79, D(G(z)): 0.16
Epoch [64/

Epoch [91/200], Step [200/600], d_loss: 0.7094, g_loss: 2.0428, D(x): 0.70, D(G(z)): 0.16
Epoch [91/200], Step [400/600], d_loss: 0.6852, g_loss: 1.8762, D(x): 0.81, D(G(z)): 0.22
Epoch [91/200], Step [600/600], d_loss: 0.7881, g_loss: 1.7452, D(x): 0.76, D(G(z)): 0.25
Epoch [92/200], Step [200/600], d_loss: 0.7752, g_loss: 1.7554, D(x): 0.73, D(G(z)): 0.23
Epoch [92/200], Step [400/600], d_loss: 0.7219, g_loss: 2.3376, D(x): 0.69, D(G(z)): 0.18
Epoch [92/200], Step [600/600], d_loss: 0.7207, g_loss: 1.6509, D(x): 0.75, D(G(z)): 0.25
Epoch [93/200], Step [200/600], d_loss: 0.8929, g_loss: 1.8734, D(x): 0.70, D(G(z)): 0.23
Epoch [93/200], Step [400/600], d_loss: 0.7131, g_loss: 1.8334, D(x): 0.84, D(G(z)): 0.32
Epoch [93/200], Step [600/600], d_loss: 0.6905, g_loss: 2.2153, D(x): 0.76, D(G(z)): 0.20
Epoch [94/200], Step [200/600], d_loss: 0.9629, g_loss: 1.0386, D(x): 0.74, D(G(z)): 0.33
Epoch [94/200], Step [400/600], d_loss: 0.7977, g_loss: 1.9278, D(x): 0.79, D(G(z)): 0.29
Epoch [94/

Epoch [121/200], Step [200/600], d_loss: 0.7497, g_loss: 2.2526, D(x): 0.70, D(G(z)): 0.21
Epoch [121/200], Step [400/600], d_loss: 0.7613, g_loss: 1.7931, D(x): 0.74, D(G(z)): 0.27
Epoch [121/200], Step [600/600], d_loss: 0.6756, g_loss: 1.9220, D(x): 0.75, D(G(z)): 0.20
Epoch [122/200], Step [200/600], d_loss: 0.8449, g_loss: 1.5871, D(x): 0.67, D(G(z)): 0.23
Epoch [122/200], Step [400/600], d_loss: 0.8839, g_loss: 1.7133, D(x): 0.70, D(G(z)): 0.26
Epoch [122/200], Step [600/600], d_loss: 1.0274, g_loss: 0.9719, D(x): 0.69, D(G(z)): 0.32
Epoch [123/200], Step [200/600], d_loss: 0.8992, g_loss: 1.3678, D(x): 0.75, D(G(z)): 0.36
Epoch [123/200], Step [400/600], d_loss: 0.6775, g_loss: 2.0386, D(x): 0.77, D(G(z)): 0.23
Epoch [123/200], Step [600/600], d_loss: 0.9056, g_loss: 1.7382, D(x): 0.69, D(G(z)): 0.29
Epoch [124/200], Step [200/600], d_loss: 0.8234, g_loss: 1.8255, D(x): 0.65, D(G(z)): 0.18
Epoch [124/200], Step [400/600], d_loss: 1.0204, g_loss: 1.7268, D(x): 0.63, D(G(z)): 0.28

Epoch [151/200], Step [200/600], d_loss: 0.8412, g_loss: 1.6670, D(x): 0.77, D(G(z)): 0.35
Epoch [151/200], Step [400/600], d_loss: 0.8747, g_loss: 1.4557, D(x): 0.65, D(G(z)): 0.21
Epoch [151/200], Step [600/600], d_loss: 1.0203, g_loss: 1.9036, D(x): 0.59, D(G(z)): 0.19
Epoch [152/200], Step [200/600], d_loss: 0.8287, g_loss: 1.5432, D(x): 0.68, D(G(z)): 0.25
Epoch [152/200], Step [400/600], d_loss: 1.0846, g_loss: 1.3091, D(x): 0.63, D(G(z)): 0.30
Epoch [152/200], Step [600/600], d_loss: 0.8192, g_loss: 1.6376, D(x): 0.76, D(G(z)): 0.32
Epoch [153/200], Step [200/600], d_loss: 0.9049, g_loss: 1.5929, D(x): 0.74, D(G(z)): 0.32
Epoch [153/200], Step [400/600], d_loss: 0.9290, g_loss: 1.4433, D(x): 0.64, D(G(z)): 0.27
Epoch [153/200], Step [600/600], d_loss: 0.8943, g_loss: 1.4881, D(x): 0.63, D(G(z)): 0.20
Epoch [154/200], Step [200/600], d_loss: 0.9352, g_loss: 1.7200, D(x): 0.70, D(G(z)): 0.30
Epoch [154/200], Step [400/600], d_loss: 0.8130, g_loss: 1.5310, D(x): 0.71, D(G(z)): 0.26

Epoch [181/200], Step [200/600], d_loss: 0.7956, g_loss: 1.6524, D(x): 0.78, D(G(z)): 0.32
Epoch [181/200], Step [400/600], d_loss: 0.8145, g_loss: 1.5121, D(x): 0.75, D(G(z)): 0.33
Epoch [181/200], Step [600/600], d_loss: 0.8498, g_loss: 1.4914, D(x): 0.67, D(G(z)): 0.26
Epoch [182/200], Step [200/600], d_loss: 0.9677, g_loss: 1.5046, D(x): 0.63, D(G(z)): 0.26
Epoch [182/200], Step [400/600], d_loss: 0.7006, g_loss: 1.9451, D(x): 0.75, D(G(z)): 0.25
Epoch [182/200], Step [600/600], d_loss: 1.1030, g_loss: 1.6648, D(x): 0.65, D(G(z)): 0.34
Epoch [183/200], Step [200/600], d_loss: 0.9557, g_loss: 1.5384, D(x): 0.71, D(G(z)): 0.34
Epoch [183/200], Step [400/600], d_loss: 0.8908, g_loss: 1.6175, D(x): 0.62, D(G(z)): 0.19
Epoch [183/200], Step [600/600], d_loss: 0.9312, g_loss: 1.4915, D(x): 0.69, D(G(z)): 0.31
Epoch [184/200], Step [200/600], d_loss: 1.1373, g_loss: 2.0298, D(x): 0.57, D(G(z)): 0.24
Epoch [184/200], Step [400/600], d_loss: 0.8822, g_loss: 1.3416, D(x): 0.66, D(G(z)): 0.26

In [9]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')